In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
from datetime import datetime

In [9]:
from datetime import datetime

def log_progress(message):
    with open('./code_log.txt', 'a') as f:
        f.write(f'{datetime.now()}: {message}.\n')

In [10]:
def extract(url, table_att):
    soup = requests.get(url).text
    web = BeautifulSoup(soup, 'html.parser')
    table = web.find('span', string=table_att).find_next('table')
    df = pd.read_html(StringIO(str(table)))[0]
    
    log_progress('Data Extracted successfully from Wikipedia')
    
    return df

In [11]:
import pandas as pd
from datetime import datetime

def log_progress(message):
    with open('./code_log.txt', 'a') as f:
        f.write(f'{datetime.now()}: {message}.\n')

def transformation(df, csv_path):
    exchange_rate = pd.read_csv(csv_path, index_col=0).to_dict()['Rate']
    
    df['MC_GBP_Billion'] = round(df['Market cap (US$ Billion)'] * exchange_rate['GBP'], 2)
    df['MC_EUR_Billion'] = round(df['Market cap (US$ Billion)'] * exchange_rate['EUR'], 2)
    df['MC_INR_Billion'] = round(df['Market cap (US$ Billion)'] * exchange_rate['INR'], 2)
    
    print(df['MC_EUR_Billion'][4])
    
    log_progress('Data Transformation done now, Step 2 Complete.')
    
    return df

In [12]:
def load_to_csv(df, out_path):
    df.to_csv(out_path, index=False)
    log_progress('Data loaded into CSV File')

In [13]:
def load_to_db(df, sql_conn, tn):
    df.to_sql(tn, sql_conn, if_exists='replace', index=False)
    log_progress('Data loaded into the SQLite Database')

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
from datetime import datetime
import sqlite3

def log_progress(message):
    with open('./code_log.txt', 'a') as f:
        f.write(f'{datetime.now()}: {message}.\n')

def extract(url, table_att):
    soup = requests.get(url).text
    web = BeautifulSoup(soup, 'html.parser')
    table = web.find('span', string=table_att).find_next('table')
    df = pd.read_html(StringIO(str(table)))[0]
    
    log_progress('Data Extracted successfully from Wikipedia')
    
    return df

def transformation(df, csv_path):
    # Print out the columns of the DataFrame to check for discrepancies
    print("Columns in the DataFrame:", df.columns.tolist())
    
    # Load exchange rates
    exchange_rate = pd.read_csv(csv_path, index_col=0).to_dict()['Rate']
    
    # Ensure the column name matches exactly
    column_name = 'Market cap (US$ Billion)'
    if column_name in df.columns:
        df['MC_GBP_Billion'] = round(df[column_name] * exchange_rate['GBP'], 2)
        df['MC_EUR_Billion'] = round(df[column_name] * exchange_rate['EUR'], 2)
        df['MC_INR_Billion'] = round(df[column_name] * exchange_rate['INR'], 2)
        
        print(df['MC_EUR_Billion'][4])
        
        log_progress('Data Transformation done now, Step 2 Complete.')
    else:
        log_progress(f"Column '{column_name}' not found in DataFrame")
        raise KeyError(f"Column '{column_name}' not found in DataFrame")
    
    return df

def load_to_csv(df, out_path):
    df.to_csv(out_path, index=False)
    log_progress('Data loaded into CSV File')

def load_to_db(df, sql_conn, tn):
    df.to_sql(tn, sql_conn, if_exists='replace', index=False)
    log_progress('Data loaded into the SQLite Database')

if __name__ == '__main__':
    url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
    output_csv_path = './Largest_banks_data.csv'
    database_name = 'Banks.db'
    table_name = 'Largest_banks'
    
    log_progress('Preliminaries complete. Initiating ETL process')
    
    # Extract data
    df = extract(url, 'By market capitalization')
    log_progress('Extraction Complete')
    print(df)
    
    # Transform data
    
    df = transformation(df, './input/exchange_rate.csv')
    log_progress('Data Transformed successfully')
    print(df)
   
    
    # Load data to CSV
    load_to_csv(df, output_csv_path)
    log_progress('Data has been transformed and loaded to CSV')
    
    # Load data to SQLite database
    with sqlite3.connect(database_name) as conn:
        load_to_db(df, conn, table_name)
    log_progress('DataFrame has been uploaded into SQLite Database successfully')

   Rank                                Bank name  Market cap (US$ billion)
0     1                           JPMorgan Chase                    432.92
1     2                          Bank of America                    231.52
2     3  Industrial and Commercial Bank of China                    194.56
3     4               Agricultural Bank of China                    160.68
4     5                                HDFC Bank                    157.91
5     6                              Wells Fargo                    155.87
6     7                        HSBC Holdings PLC                    148.90
7     8                           Morgan Stanley                    140.83
8     9                  China Construction Bank                    139.82
9    10                            Bank of China                    136.81
Columns in the DataFrame: ['Rank', 'Bank name', 'Market cap (US$ billion)']


KeyError: "Column 'Market cap (US$ Billion)' not found in DataFrame"